In [1]:
import os

In [2]:
%pwd

'c:\\Users\\asus4\\OneDrive\\Documents\\DEEP LEARNING PROJECT\\DL-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\asus4\\OneDrive\\Documents\\DEEP LEARNING PROJECT\\DL-Project'

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass 
from pathlib import Path 

@dataclass
class EvaluationConfig:
    path_of_model: Path
    all_params: dict
    train_data_path: Path 
    param_batch_size: int
    param_image_size: list

In [9]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories,save_json

In [10]:
class ConfigManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        evaluation_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            all_params=self.params,
            train_data_path="artifacts/data_ingestion/Chicken fickle images",
            param_batch_size=self.params.params_batch_size,
            param_image_size=self.params.params_image_size
        )
        return evaluation_config

In [11]:
from urllib.parse import urlparse

In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
          
    def _valid_generator(self):
        datagen_kwargs = dict(rescale=1./255,validation_split=0.30) 
        dataflow_kwargs = dict(
            target_size=self.config.param_image_size[:-1],
            batch_size=self.config.param_batch_size,
            interpolation="bilinear"
        )

        valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs) 

        self.valid_generator = valid_datagen.flow_from_directory(
            directory=self.config.train_data_path, 
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def _load_model(path_of_model: Path)-> tf.keras.Model:
        model = tf.keras.models.load_model(path_of_model)
        return model 

    def evaluate_model(self):
        self.model = self._load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator) 

    def save_score(self): 
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path = Path("scores.json"),data =  scores)

In [13]:
try: 
    config_manager = ConfigManager()
    evaluation_config = config_manager.get_evaluation_config()

    evaluation = Evaluation(config=evaluation_config)
    evaluation.evaluate_model()
    evaluation.save_score() 
except Exception as e:
    raise e

Found 116 images belonging to 2 classes.


c:\Users\asus4\OneDrive\Documents\DEEP LEARNING PROJECT\DL-Project\chicken\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 800ms/step - accuracy: 0.8621 - loss: 3.3944
